In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow import keras

2025-04-24 02:34:48.104985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745454888.177342    5286 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745454888.196349    5286 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-24 02:34:48.325611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# loading data and preprocessing

In [2]:
INPUT_SIZE = 784
HIDDEN_SIZE = 32
OUTPUT_SIZE = 10

from keras.datasets import mnist


(x_train,y_train),(x_test, y_test) = mnist.load_data()
print("images train before normalize and processing",x_train.shape)

x_train = tf.reshape(x_train, [-1, 28*28])
x_test = tf.reshape(x_test, [-1, 28*28])

print("images train after reshaping",x_train.shape)
# x_train[1]

# handling pixels to be only black and white from gray scale
x_train = tf.cast(x_train, tf.float32) / 255.0
x_test = tf.cast(x_test, tf.float32) / 255.0


y_train_hot = keras.utils.to_categorical(y_train, num_classes=10)
y_test_hot = keras.utils.to_categorical(y_test, num_classes=10)
print(y_train_hot.shape)
print(y_test_hot.shape)

y_train_hot = tf.convert_to_tensor(y_train_hot, dtype=tf.float32)
y_test_hot = tf.convert_to_tensor(y_test_hot, dtype=tf.float32)



images train before normalize and processing (60000, 28, 28)
images train after reshaping (60000, 784)
(60000, 10)
(10000, 10)


I0000 00:00:1745454892.449730    5286 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2169 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


# neural netowrk structure

In [3]:
INPUT_SIZE = 784
HIDDEN_SIZE = 32
OUTPUT_SIZE = 10

def sigmoid(x):
    return tf.nn.sigmoid(x)

def softmax(x):
    return tf.nn.softmax(x)

w1_size = INPUT_SIZE * HIDDEN_SIZE
b1_size = HIDDEN_SIZE
w2_size = HIDDEN_SIZE * OUTPUT_SIZE
b2_size = OUTPUT_SIZE
D = w1_size + b1_size + w2_size + b2_size


def decode_weights(vector):
    
    w1_size = INPUT_SIZE * HIDDEN_SIZE
    b1_size = HIDDEN_SIZE
    w2_size = HIDDEN_SIZE * OUTPUT_SIZE
    b2_size = OUTPUT_SIZE
    
    idx = 0
    w1 = tf.reshape(vector[idx:idx + w1_size], [INPUT_SIZE, HIDDEN_SIZE])
    idx += w1_size
    
    b1 = vector[idx:idx + b1_size]
    idx += b1_size
    
    w2 = tf.reshape(vector[idx:idx + w2_size], [HIDDEN_SIZE, OUTPUT_SIZE])
    idx += w2_size
    
    b2 = vector[idx:idx + b2_size]
    
    return w1, b1, w2, b2


def forward_pass(X, vector):
    w1, b1, w2, b2 = decode_weights(vector)
    
    hidden = sigmoid(tf.matmul(X, w1) + b1)
    output = softmax(tf.matmul(hidden, w2) + b2)
    
    return output

def fitness(vector, X, y_true, batch_size=1000):
    
    indices = np.random.choice(X.shape[0], batch_size, replace=False)
    X_batch = tf.gather(X, indices)
    y_batch = tf.gather(y_true, indices)
    
    y_pred = forward_pass(X_batch, vector)
    loss = tf.keras.losses.categorical_crossentropy(y_batch, y_pred)
    return tf.reduce_mean(loss)
    
# def fitness(vector, X, y_true):
#     y_pred = forward_pass(X, vector)
    
#     loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    
#     return tf.reduce_mean(loss)



In [4]:
import numpy as np
total_weights = INPUT_SIZE * HIDDEN_SIZE + HIDDEN_SIZE + HIDDEN_SIZE * OUTPUT_SIZE + OUTPUT_SIZE
vector_np = np.random.randn(total_weights) * 0.01
vector_tf = tf.convert_to_tensor(vector_np, dtype=tf.float32)

loss = fitness(vector_tf, x_train, y_train_hot)
print(f"Initial loss: {loss.numpy()}")


Initial loss: 2.3042843341827393


# functions for the DE

In [5]:
import numpy as np

np.random.seed(24)
tf.random.set_seed(24)

def initialize_population(population_size, D, min_val=-0.5, max_val=0.5):
    

    population = tf.random.uniform(
        shape=(population_size, D),
        minval=min_val,
        maxval=max_val,
        dtype=tf.float32
    )
    
    return population


    


In [6]:
np.random.seed(24)
tf.random.set_seed(24)
def random_vectors(population, current_index):

    if isinstance(population, tf.Tensor):  # converts from tf to np
        population_np = population.numpy()
    else:
        population_np = population
    
    population_size = population_np.shape[0]
    
    available_indices = np.delete(np.arange(population_size), current_index)
    
    selected_indices = np.random.choice(available_indices, size=3, replace=False)
    
    return (population_np[selected_indices[0]], 
            population_np[selected_indices[1]], 
            population_np[selected_indices[2]])


    

In [7]:
NP = 100

population = initialize_population(population_size=NP ,D=D)
print("population shape is : ",population.shape)

v1, v2, v3 = random_vectors(population, 5)

v1_tf = tf.convert_to_tensor(v1)
v2_tf = tf.convert_to_tensor(v2)
v3_tf = tf.convert_to_tensor(v3)

population shape is :  (100, 25450)


In [8]:
v1_tf.shape

TensorShape([25450])

In [9]:
def mutate(x1, x2, x3, F, D, L=-1, H=1):

    v = x3 + F * (x1 - x2)
    
    # ocnstrain handling
    out_of_bounds = tf.logical_or(v < L, v > H) # returns tensor fo booleans where true is erorr
    
    random_values = L + tf.random.uniform(shape=(D,), dtype=tf.float32) * (H - L)
    
    v = tf.where(out_of_bounds, random_values, v) # checks where true and takes value from the random value if false takes from v
    
    return v

 <p>example of how we did the mutate and how mask works</p>
<img src="Screenshot_20250424_015153.png" alt="Project Screenshot" width="600"/>

In [10]:
v = mutate(v1_tf, v2_tf, v3_tf, 0.9, D)
print(v.shape)
print(v)

(25450,)
tf.Tensor(
[-0.17597383  0.33066928 -0.13281165 ...  0.65596056 -0.09411368
  0.33964458], shape=(25450,), dtype=float32)


In [11]:
def crossover(target_vector, mutant_vector, CR, D):

    # instead of loop and check we make the whole vecotor once and check with mask
    r = tf.random.uniform(shape=(D,), dtype=tf.float32)
    
    
    mask = r < CR  # mask is tensor flow boolean vecotr
    
    trial_vector = tf.where(mask, mutant_vector, target_vector)
    
    return trial_vector

# main evolve function

In [12]:
def evolve(population, GEN, NP, D, F, CR, x_train, y_train, L=-0.5, H=0.5):

    best_solution = None
    best_overall_fitness = float('inf')  # infinity for minimization
    
    
    for g in range(GEN):
        
        for j in range(NP):
            
            v1, v2, v3 = random_vectors(population, j)
            

            v1 = tf.convert_to_tensor(v1, dtype=tf.float32)
            v2 = tf.convert_to_tensor(v2, dtype=tf.float32)
            v3 = tf.convert_to_tensor(v3, dtype=tf.float32)
        
            # mutation
            mutant_child = mutate(v1, v2, v3, F, D, L, H)
            
            target_vector = population[j]
            if not isinstance(target_vector, tf.Tensor):
                target_vector = tf.convert_to_tensor(target_vector, dtype=tf.float32)
            
            # crossover
            trial_vector = crossover(target_vector, mutant_child, CR, D)
            
            # fitness
            trial_fitness = fitness(trial_vector, x_train, y_train  )
            target_fitness = fitness(target_vector,x_train, y_train )
            
        
            if trial_fitness < target_fitness:  
                population[j].assign(trial_vector)  
        
        best_fitness = float('inf')
        best_index = 0
        for i in range(NP):
            current_fitness = fitness(population[i],x_train, y_train )
            if current_fitness < best_fitness:  
                best_fitness = current_fitness
                best_index = i
                
            if current_fitness < best_overall_fitness:  
                best_overall_fitness = current_fitness
                best_solution = tf.identity(population[i])  
                
        print(f"Generation {g}: Best fitness (loss) = {best_fitness}")
    
    return best_solution

In [ ]:
population = tf.Variable(population)
result = evolve(population, 100, NP, D, 0.8, 0.7, x_train, y_train_hot, L=-1, H=1)


Generation 0: Best fitness (loss) = 2.3856887817382812
Generation 1: Best fitness (loss) = 2.3434674739837646
Generation 2: Best fitness (loss) = 2.397263288497925
Generation 3: Best fitness (loss) = 2.4033985137939453
Generation 4: Best fitness (loss) = 2.401360034942627
Generation 5: Best fitness (loss) = 2.4251039028167725
Generation 6: Best fitness (loss) = 2.3919858932495117
Generation 7: Best fitness (loss) = 2.3759944438934326
Generation 8: Best fitness (loss) = 2.3865060806274414
Generation 9: Best fitness (loss) = 2.4070234298706055
Generation 10: Best fitness (loss) = 2.404425859451294
Generation 11: Best fitness (loss) = 2.406125545501709
Generation 12: Best fitness (loss) = 2.4003679752349854
Generation 13: Best fitness (loss) = 2.4121792316436768
Generation 14: Best fitness (loss) = 2.4267470836639404
Generation 15: Best fitness (loss) = 2.385441541671753
Generation 16: Best fitness (loss) = 2.383615732192993
Generation 17: Best fitness (loss) = 2.387791395187378
Generatio

In [ ]:
def evaluate_model(best_solution, x_test, y_test):

    # Get predictions
    probabilities = forward_pass(x_test, best_solution)
    predictions = tf.argmax(probabilities, axis=1)
    true_labels = tf.argmax(y_test, axis=1)
    
    # Calculate accuracy
    correct = tf.equal(predictions, true_labels)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    # Calculate loss
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_test, probabilities))
    
    print(f"Test accuracy: {accuracy.numpy() * 100:.2f}%")
    print(f"Test loss: {loss.numpy():.4f}")
    
    return accuracy.numpy(), predictions.numpy(), probabilities.numpy()

In [ ]:
acc, pred , prob = evaluate_model(result, x_test,y_test_hot)

In [ ]:
def visualize_predictions(x_test, y_test, predictions, probabilities, num_samples=5):
    """
    Visualize sample predictions from the model
    
    Args:
        x_test: Test input data
        y_test: Test labels (one-hot encoded)
        predictions: Predicted classes
        probabilities: Prediction probabilities
        num_samples: Number of samples to visualize
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import random
    
    # Get random samples
    indices = random.sample(range(len(x_test)), num_samples)
    
    # Set up the figure
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 2*num_samples))
    
    for i, idx in enumerate(indices):
        # Original image
        original_image = x_test[idx].numpy().reshape(28, 28)
        true_label = np.argmax(y_test[idx])
        pred_label = predictions[idx]
        pred_prob = probabilities[idx][pred_label]
        
        # Show image
        axes[i, 0].imshow(original_image, cmap='gray')
        axes[i, 0].set_title(f"True: {true_label}")
        axes[i, 0].axis('off')
        
        # Show probabilities
        axes[i, 1].bar(range(10), probabilities[idx])
        axes[i, 1].set_title(f"Pred: {pred_label} (Confidence: {pred_prob:.4f})")
        axes[i, 1].set_xticks(range(10))
        
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_predictions(x_test, y_test_hot,pred,prob,10)